In [1]:
import pandas as pd
import numpy as np

import src.latent_factor_xai as lfxai
import concepts

In [2]:
def getMoviesExplanation(recommendation, lattice):
    """
        Devuelve la lista de peliculas que comparten el nodo padre con la pelicula recomendada
    """
    
    # el nodo padre es aquel (o aquellos nodos) en el que encontramos el mayor numero de atributos para la pelicula recomendada
    # al ser el nodo mas especifico, es el nodo en el que la recomendacion se encuentra que mas atributos tiene 
    # por tanto tenemos que encontrar ese intent (el que tenga mayor numero de atributos donde se encuentra la pelicula 
    # recomendada en el extent, pero cuando esa peli recomendada no este sola) y coger todos los extent de ese nodo
    
    my_movies = list()
    
    # guardamos todos las movies con sus atributos compartidos segun el reticulo
    for extent, intent in lattice:
        my_movies.append((extent,intent)) # movies, attributes
    
    # de la lista total, nos quedamos con aquellos conjuntos de peliculas donde este la recomendacion, y donde 
    # la recomendacion no este sola, porque no tiene sentido
    my_movies_2 = [(x[0],x[1],len(x[1])) for x in my_movies if recommendation in x[0] and len(x[0]) > 1]
    
    # guardamos la lista donde haya mas atributos compartidos
    final_list = list(max(my_movies_2,key=lambda item:item[2])[0])
    
    # de la lista, eliminamos la recomendacion 
    final_list.remove(recommendation)
    
    return final_list

In [3]:
def get_dummie(df, column, sep):
    new_df = df[column].str.get_dummies(sep=sep)
    result = pd.concat([df, new_df], axis=1)        
    #result.drop(columns=[column])
    return result

In [4]:
def dataframe_to_context_matrix(lattice_movies):
    # Generamos la matriz necesaria para concepts
    lattice_movies['title_year'] = lattice_movies['title_year'].apply(lambda val: str(val)) # pasamos años a str
    lista_columns = ['director_name', 'genres', 'stars', 'language', 'country', 'title_year']

    for c in range(len(lista_columns)):
        lattice_movies = get_dummie(lattice_movies, lista_columns[c], sep='|')

    lista_columns_to_drop = ['director_name', 'genres', 'stars', 'language', 'country', 'title_year', 'movie_title', 'duration']
    lattice_movies.drop(columns=lista_columns_to_drop, axis=1, inplace=True)

    result = lattice_movies.replace([0, 1], ['', 'X'])
    return result.set_index(['id'])

In [5]:
def get_lattice(movie_recommended, examples):
    movies_ids = np.append(examples, movie_recommended)
    
    # Obtengo las descripciones de las películas
    lattice_val = movies_attr_df[movies_attr_df['id'].isin(lattice_ids)]
    
    # Lo convierto a una matriz válida para concepts
    lattice_val = dataframe_to_context_matrix(lattice_val)
    
    objects = lattice_movies.index.tolist()
    properties = list(lattice_movies)
    bools = list(lattice_movies.fillna(False).astype(bool).itertuples(index=False, name=None))

    return concepts.Context(objects, properties, bools)

## Prueba Concepts

Para cargar un retículo, es necesario primero crear un DF donde el índice sea el id de la película, las columnas sean los atributos y los valores sean vacíos o X.

In [6]:
# Cargamos los datos
trainset = pd.read_csv('data/experiment_data/trainset.csv')
testset = pd.read_csv('data/experiment_data/testset.csv')
movies_attr_df = pd.read_csv('data/experiment_data/movies.csv')

In [7]:
# Creamos y entrenamos el modelo
model = lfxai.NMF_XAI()
model.fit(trainset, movies_attr_df)

In [8]:
# Ejemplo de ejecución
user = 12
movie = 14

In [9]:
# Obtenemos los ejemplos de explicación
pred = model.predict(user, movie)
examples = model.get_examples(user, movie)

In [10]:
# Obtenemos las descripciones de todas las películas
lattice_ids = np.append(examples, movie)
lattice_movies = movies_attr_df[movies_attr_df['id'].isin(lattice_ids)]

In [11]:
# Generamos la matriz necesaria para concepts
lattice_movies['title_year'] = lattice_movies['title_year'].apply(lambda val: str(val)) # pasamos años a str
lista_columns = ['director_name', 'genres', 'stars', 'language', 'country', 'title_year']

for c in range(len(lista_columns)):
    lattice_movies = get_dummie(lattice_movies, lista_columns[c], sep='|')
    
lista_columns_to_drop = ['director_name', 'genres', 'stars', 'language', 'country', 'title_year', 'movie_title', 'duration']
lattice_movies.drop(columns=lista_columns_to_drop, axis=1, inplace=True)

lattice_movies = lattice_movies.replace([0, 1], ['', 'X'])

/var/folders/n4/jgn7dyds78s1nj91t_rr3xnc0000gn/T/ipykernel_14024/3312553073.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lattice_movies['title_year'] = lattice_movies['title_year'].apply(lambda val: str(val)) # pasamos años a str


In [12]:
# Generamos el retículo
lattice_movies.set_index(['id'], inplace=True)
objects = [str(x) for x in lattice_movies.index.tolist()]
properties = list(lattice_movies)
bools = list(lattice_movies.fillna(False).astype(bool).itertuples(index=False, name=None))

context_movies = concepts.Context(objects, properties, bools)

In [13]:
for extent, intent in context_movies.lattice:
    print("{} - {}".format(extent, intent))

() - ('Amy Heckerling', 'Gillian Armstrong', 'Jerry Zucker', 'Oliver Stone', 'Action', 'Adventure', 'Biography', 'Comedy', 'Drama', 'Family', 'History', 'Romance', 'Thriller', 'Anthony Hopkins', 'Christian Bale', 'Donald Faison', 'Elisa Donovan', 'Eric Stoltz', 'Joan Allen', 'John Gielgud', 'Julia Ormond', 'English', 'USA', '1994', '1995')
('261',) - ('Gillian Armstrong', 'Drama', 'Family', 'Romance', 'Christian Bale', 'Eric Stoltz', 'English', 'USA', '1994')
('39',) - ('Amy Heckerling', 'Comedy', 'Romance', 'Donald Faison', 'Elisa Donovan', 'English', 'USA', '1995')
('168',) - ('Jerry Zucker', 'Action', 'Adventure', 'Romance', 'Thriller', 'John Gielgud', 'Julia Ormond', 'English', 'USA', '1995')
('14',) - ('Oliver Stone', 'Biography', 'Drama', 'History', 'Anthony Hopkins', 'Joan Allen', 'English', 'USA', '1995')
('261', '14') - ('Drama', 'English', 'USA')
('39', '168') - ('Romance', 'English', 'USA', '1995')
('261', '39', '168') - ('Romance', 'English', 'USA')
('39', '168', '14') - ('

In [14]:
getMoviesExplanation(movie, context_movies.lattice)

ValueError: max() arg is an empty sequence